## The purpose of this script is to display the stocks/equities for the Maltese market. The original data was sourced from the Borza ta Malta and contains stock market data for the past 10 years.  

In [ ]:
!pip install pyngrok
!pip install voila

### Imports

In [ ]:
import plotly.graph_objects as go

import pandas as pd
from   datetime import datetime

from   ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from pyngrok import ngrok

In [ ]:
ALL_OPTION = "ALL"

In [ ]:
def get_data():
  #Read data file
  df_sheet1 = pd.read_excel("/content/TradingStats.xlsx",sheet_name=0)
  df_sheet2 = pd.read_excel("/content/TradingStats.xlsx",sheet_name=1)
  df = pd.concat([df_sheet1,df_sheet2])

  df.columns = ['symbol','high', 'low', 'open_price', 'close_price','change_cents', 'twap', 'deals', 'volume', 'value','code','date']
  df['year'] = df['date'].dt.year

  df.set_index("date",inplace=True)

  return df

In [ ]:
def prepare_widets(df):

  #years dropdown
  list_years = list(df['year'].unique())
  list_years = sorted(list_years,reverse=True)
  options    = [(entity,entity) for entity in list_years if entity is not None  ]
  options    = [(ALL_OPTION,ALL_OPTION)] + options

  years_dropdown = widgets.SelectMultiple(
      options    = options,
      #value     = [ALL_OPTION],
      index      = [1],
      description= 'Years',
      disabled   =  False
  )

  #date range
  options = [(date.strftime(' %d %b %Y '), date) for date in df.index]
  index = (0, len(options)-1)
  date_selection_range_slider = widgets.SelectionRangeSlider(
      options=options,
      index=index,
      description='Dates',
      orientation='horizontal',
      layout={'width': '500px'}
  )

  #Frequency
  options=[('Day',None),('Month','Month'),('Year','Year')]
  freq_dropdown = widgets.Dropdown(
      options=options,
      value='Month',
      description='Frequency:',
      disabled=False,
  )

  #symbols
  list_sym = list(df['symbol'].unique())
  list_sym.sort()

  options = [(sym,sym) for sym in list_sym if sym is not None  ]
  options = [(ALL_OPTION,ALL_OPTION)] + options

  symbols_dropdown = widgets.SelectMultiple(
      options=options,
      #index = [0],
      value = ["BOV"],
      #rows=10,
      description='Symbols',
      disabled=False
  )

  w = {"symbols_widget":symbols_dropdown, 
       "years_widget":years_dropdown,
       "frequency_widget": freq_dropdown,
       #"date_range":date_selection_range_slider
       }
  
  ui = widgets.VBox(list(w.values()))

  def on_value_change(change):
    selected_years = list(change['new'])
    new_options = df['year'].isin(selected_years).index.unique().strftime("%Y-%m-%d").to_list()
    date_selection_range_slider.options = new_options

  #years_dropdown.observe(on_value_change, names='value')

  return w,ui

def resample_dataframe(df,rule,rounding_digits=3):
  rule = rule.lower()
  rule_map = {"year":"YS","month":"MS","day":"D"}

  if not rule in rule_map:
    raise ValueError("Invalid frequency rule. Must be {}".format(rule_map.keys()))

  cols         = ['high', 'low', 'open_price', 'close_price']
  df_tmp       = df.groupby(by=['symbol','year']).resample(rule_map[rule])[cols].mean().reset_index()
  df_tmp[cols] = df_tmp[cols].round(rounding_digits)
  df_tmp       = df_tmp.set_index("date")

  return df_tmp


def plot_cs(df,symbols,years=None,frequency=None):

  if frequency is not None:
    df = resample_dataframe(df,frequency)

  #Filter by symbol 
  if symbols is not None and symbols[0] != ALL_OPTION:
    df = df[df['symbol'].isin(list(symbols))]

  #Filter by years
  if years is not None and years[0] != ALL_OPTION:
    df = df[df['year'].isin(list(years))]

  #Plot candlestick
  fig = go.Figure(data=[go.Candlestick(
                x    =df.index,
                open =df['open_price'],
                high =df['high'],
                low  =df['low'],
                close=df['close_price'])])
  
  fig.update_layout(title='Stock for {}, years: {}, frequency: {}'.format(list(symbols),list(years),frequency),
                    yaxis_title='Stock',margin=dict(l=20, r=20, t=20, b=20),autosize=False,width=1700,height=500
                    )

  fig.show()


## Main

In [ ]:
df = get_data()
df.head()

,symbol,high,low,open_price,close_price,change_cents,twap,deals,volume,value,code,year
date,,,,,,,,,,,,
2010-01-04,BOV,3.900,3.87,3.88,3.900,0.030,3.897,19,23455,91398.236,510,2010
2010-01-04,CW,0.400,0.40,0.40,0.400,0.010,0.400,1,40000,16000.000,510,2010
2010-01-04,GO,2.100,2.10,2.10,2.100,0.000,2.100,1,2000,4200.000,510,2010
2010-01-04,HSB,3.249,3.23,3.23,3.249,0.019,3.238,6,2587,8375.675,510,2010
2010-01-04,IHI,0.800,0.80,0.80,0.800,0.000,0.800,2,15000,12000.000,510,2010


In [ ]:
#UI
ui_items,ui = prepare_widets(df)

#Plotting
out = widgets.interactive_output(plot_cs,{'df'        :fixed(df),
                                          'symbols'   :ui_items['symbols_widget'],
                                          'years'     :ui_items['years_widget'],
                                          'frequency' :ui_items['frequency_widget']
                                          })

display(ui,out)

Output()

In [ ]:
!ngrok authtoken 1tTnoZe4lrUjDPx81Nc0Hz4H1Y5_5xhW8Ggkr2q3LKC5inVLE
!voila --no-browser /content/Trading.ipynb &

In [ ]:
public_url = ngrok.connect(port=8866)
public_url

<NgrokTunnel: "http://55773785c766.ngrok.io" -> "http://localhost:80">